In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import joblib

In [ ]:
tr_tr_new = joblib.load('joblib/tr_tr_encoded.joblib')

In [ ]:
te_tr_new = joblib.load('joblib/te_tr_encoded.joblib')

In [ ]:
from sklearn.model_selection import train_test_split
X = tr_tr_new.drop('isFraud', axis=1)
y = tr_tr_new['isFraud']

In [ ]:
import numpy as np
# Import necessary libraries
from catboost import CatBoostClassifier, Pool

# Load your data (assuming you're using pandas and your data is in dataframe format)
# import pandas as pd
# train_data = pd.read_csv('path_to_train_data.csv')
# test_data = pd.read_csv('path_to_test_data.csv')

# For the sake of this example, let's assume you have `tr_tr_new` as your training data
# and `te_tr_new` as your testing data, with 'target' as the target column.
X_train = tr_tr_new.drop('target', axis=1)
y_train = tr_tr_new['target']

X_test = te_tr_new.drop('target', axis=1)
y_test = te_tr_new['target']

# Identify categorical features. CatBoost can handle them natively.
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=500, # or any other desired value
                           learning_rate=0.05, # or any other desired value
                           depth=7, # or any other desired value
                           cat_features=categorical_features_indices,
                           verbose=100) # set verbose to your desired value or remove for silent training

# Train model
model.fit(X_train, y_train)

# Predict
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)

# Evaluate model (e.g., using accuracy for classification tasks)
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, preds_class))
